#### <center><h2>Artificial and Computational Intelligence (Assignment - 2)</h2></center>

## Problem Statement

As part of the 2nd Assignment, we'll implement Bayesian Networks and also learn to use the pomegranate library.

You are required to create a bayesian network model which would help you predict the probability. The detailed problem description is attached as a PDF as a part of this assignment along with the marking scheme.  

### What is a Bayesian Network ?

A Bayesian network, Bayes network, belief network, decision network, Bayes(ian) model or probabilistic directed acyclic graphical model is a probabilistic graphical model (a type of statistical model) that represents a set of variables and their conditional dependencies via a directed acyclic graph (DAG). 

Bayesian networks are ideal for taking an event that occurred and predicting the likelihood that any one of several possible known causes was the contributing factor. For example, a Bayesian network could represent the probabilistic relationships between diseases and symptoms. Given symptoms, the network can be used to compute the probabilities of the presence of various diseases. 

### Dataset

The dataset can be downloaded from https://drive.google.com/drive/folders/1oMtKmmvPkN4O8DmrHMJe6M8CbB93Z5kw .You can access it only using your BITS IDs. Also, the same dataset is attached along with the assignment. 

#### Dataset Description
##### Sample Tuple

Y	won	5wickets	lost	2nd	vWest_Indies	Home	6-Nov-11

##### Explanation
- The first column represents if Ashwin was in the playing 11 or not. 
- The second column represents the Result of the match . win indicates India won the match.
- The third column represents the Margin of victory / losss.
- The fourth column represents the results of the toss. won indicates India won the toss. 
- The fifth column represents the batting order. If India batted 1st or 2nd. 
- The sixth column represents the opponent.
- The seventh column represents the location of the match. If the match was held in Home(India) or away. 
- The last column represents the start date of the match.


### Evaluation
We wish to evaluate based on 
- coding practices being followed
- commenting to explain the code and logic behind doing something
- your understanding and explanation of data
- how good the model would perform

### BITS Roll Number , Name
2018ab04633 - Jaise K Joseph

2018ab04637 - Voruganti Achyuth Reddy

2018ab04684 - Kiran Kumar K

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy
import pandas as pd

from pomegranate import *


In [2]:
#Reading data from csv file

TestStats = pd.read_csv("India_Test_Stats.csv") 

In [3]:
#Checking data

TestStats

,Ashwin,Result,Margin,Toss,Bat,Opposition,Location,Start Date
0,Y,won,5 wickets,lost,2nd,v West Indies,Home,6-Nov-11
1,Y,won,inns & 15 runs,won,1st,v West Indies,Home,14-Nov-11
2,Y,draw,-,lost,2nd,v West Indies,Home,22-Nov-11
3,Y,lost,122 runs,lost,2nd,v Australia,Away,26-Dec-11
4,Y,lost,inns & 68 runs,won,1st,v Australia,Away,3-Jan-12
...,...,...,...,...,...,...,...,...
80,Y,won,203 runs,won,1st,v South Africa,Home,2-Oct-19
81,Y,won,inns & 137 runs,won,1st,v South Africa,Home,10-Oct-19
82,Y,won,inns & 202 runs,won,1st,v South Africa,Home,19-Oct-19
83,Y,won,inns & 130 runs,lost,2nd,v Bangladesh,Home,14-Nov-19


In [4]:
TestStats.count()

Ashwin        85
Result        85
Margin        85
Toss          85
Bat           85
Opposition    85
Location      85
Start Date    85
dtype: int64

### Fuctions used

In [5]:
#Function to calcuate probability given the column and the value to find probability

def Prior_Probability(pColumns,Pvalue):
    a = list(TestStats[pColumns].unique())
    c =list(TestStats[pColumns ].value_counts())
    p=c[a.index(Pvalue)]/ TestStats.shape[0]
    return p

In [6]:
#Function to get the probabilty distribution in a dictonary  

def Prior_probabilty_distribution(PColumn):
    Prior_Probability_Dict ={}
    a = list(TestStats[PColumn].unique())
    for i in a:
        tempdict= {i:Prior_Probability(PColumn,i)}
        Prior_Probability_Dict.update(tempdict)
    return Prior_Probability_Dict

In [7]:
#Function to calculate probability given datafame, column and values

def probability(Datframe,pColumns,Pvalue):
    a = list(Datframe[pColumns].unique())
    if Pvalue not in a:
        p = 0
    else:
        c =list(Datframe[pColumns ].value_counts())
        p=c[a.index(Pvalue)]/ Datframe.shape[0]
    return p

In [8]:
#Function to conver flat list to list of list

def divide_chunks(l, n):  
    # looping till length l
    for i in range(0, len(l), n):  
        yield l[i:i + n]

In [9]:
Prior_probabilty_distribution("Result")

{'won': 0.5529411764705883,
 'draw': 0.25882352941176473,
 'lost': 0.18823529411764706}

In [10]:
Prior_probabilty_distribution("Bat")

{'2nd': 0.5411764705882353, '1st': 0.4588235294117647}

In [11]:
#Function to return conditional probability give the 2 or 3 attribute

def New_conditional_probability(v_df,v_lsit):
    lst=[]
    if len(v_lsit)==2:
        n = 3
        a=(v_df[v_lsit[0]].unique())
        b=v_df[v_lsit[1]].unique()
        for i in a:
            for j in b:
                #print(i,j)
                lk = v_df.loc[v_df[v_lsit[0]] == i]
                lst.append(i)
                lst.append(j)
                lst.append(probability(lk,v_lsit[1],j)*probability(TestStats,v_lsit[0],i))
    else:
        n = 4
        a=(v_df[v_lsit[0]].unique())
        b=v_df[v_lsit[1]].unique()
        c=v_df[v_lsit[2]].unique()
        for i in a:
            for j in b:
                lk = v_df.loc[v_df[v_lsit[0]] == i]
                for k in c: 
                    jk=lk.loc[lk[v_lsit[1]]==j]
                    lst.append(i)
                    lst.append(j)
                    lst.append(k)
                    lst.append(probability(lk,v_lsit[1],j)*probability(TestStats,v_lsit[0],i)*probability(jk,v_lsit[2],k))
    x = list(divide_chunks(lst, n))
    return x

In [12]:
#Contional probabilty of Result, Ashwin playing and location

New_conditional_probability(TestStats,["Location","Ashwin","Result"])

[['Home', 'Y', 'won', 0.38823529411764707],
 ['Home', 'Y', 'draw', 0.08235294117647059],
 ['Home', 'Y', 'lost', 0.03529411764705882],
 ['Home', 'N', 'won', 0.0],
 ['Home', 'N', 'draw', 0.0],
 ['Home', 'N', 'lost', 0.0],
 ['Away', 'Y', 'won', 0.07058823529411765],
 ['Away', 'Y', 'draw', 0.10588235294117648],
 ['Away', 'Y', 'lost', 0.1411764705882353],
 ['Away', 'N', 'won', 0.03529411764705883],
 ['Away', 'N', 'draw', 0.058823529411764705],
 ['Away', 'N', 'lost', 0.0823529411764706]]

In [13]:
#Lists for which conditional probabilty has to be calculated

Resultlist = ["Ashwin","Bat","Result"]
Ashwinlist= ["Location","Ashwin"]
Baltlist = ["Toss","Bat"]

In [14]:
#Calculating Prior pobability for Toss and Location
v_toss = DiscreteDistribution(Prior_probabilty_distribution("Toss"))
v_loc = DiscreteDistribution(Prior_probabilty_distribution("Location"))


In [15]:
#Calcualating conditional probability for Ashwin and Bat
v_ashwin = ConditionalProbabilityTable(New_conditional_probability(TestStats,Ashwinlist), [v_loc])
v_bat = ConditionalProbabilityTable(New_conditional_probability(TestStats,Baltlist),[v_toss])

In [16]:
#Calcualating conditional probability for Ashwin ,Bat and Results

x = New_conditional_probability(TestStats,Resultlist)
v_result= ConditionalProbabilityTable(x,[v_ashwin,v_bat])

## Question 3

### Model Building 

In [17]:
#Defining states

s1 = State(v_loc, 'Location')
s2 = State(v_toss, 'Toss')
s3 = State(v_ashwin, 'Ashwin')
s4 = State(v_bat, 'Bat')
s5 = State(v_result, 'Result')

In [18]:
#Getting the model

model = BayesianNetwork('Test_prediction')

In [19]:
#Adding the states to model

model.add_states(s1, s2, s3,s4,s5)

In [20]:
#Adding edges 

model.add_edge(s1,s3)
model.add_edge(s2,s4)
model.add_edge(s3,s5)
model.add_edge(s4,s5)

In [21]:
#Baking Model

model.bake()

## Question 1
Create a function to calculate prior probability of any given variable. The function should read in an array and output a dictionary of prior probability of each possible outcome. 

In [22]:
for col in ['Ashwin','Result','Toss','Bat','Location']:
    print(col,Prior_probabilty_distribution(col))

Ashwin {'Y': 0.8235294117647058, 'N': 0.17647058823529413}
Result {'won': 0.5529411764705883, 'draw': 0.25882352941176473, 'lost': 0.18823529411764706}
Toss {'lost': 0.5294117647058824, 'won': 0.47058823529411764}
Bat {'2nd': 0.5411764705882353, '1st': 0.4588235294117647}
Location {'Home': 0.5058823529411764, 'Away': 0.49411764705882355}


## Question 2
Create a function to calculate conditional probability. The function should read in multiple arrays and calculate the posterior probability of the last array wrt to previous arrays. 

In [23]:
Cond_list1 = ["Location","Ashwin"]
New_conditional_probability(TestStats,Cond_list1)

[['Home', 'Y', 0.5058823529411764],
 ['Home', 'N', 0.0],
 ['Away', 'Y', 0.31764705882352945],
 ['Away', 'N', 0.17647058823529413]]

In [24]:
Cond_list2 = ["Location","Ashwin","Result"]
New_conditional_probability(TestStats,Cond_list2)

[['Home', 'Y', 'won', 0.38823529411764707],
 ['Home', 'Y', 'draw', 0.08235294117647059],
 ['Home', 'Y', 'lost', 0.03529411764705882],
 ['Home', 'N', 'won', 0.0],
 ['Home', 'N', 'draw', 0.0],
 ['Home', 'N', 'lost', 0.0],
 ['Away', 'Y', 'won', 0.07058823529411765],
 ['Away', 'Y', 'draw', 0.10588235294117648],
 ['Away', 'Y', 'lost', 0.1411764705882353],
 ['Away', 'N', 'won', 0.03529411764705883],
 ['Away', 'N', 'draw', 0.058823529411764705],
 ['Away', 'N', 'lost', 0.0823529411764706]]

## Question 4
Use the Bayesian Network model created to calculate the probability of

#### a. India winning, batting 2nd, Ashwin playing

In [25]:
print(model.predict_proba([None, None, 'Y', '2nd', 'won']))

[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.6198457928260142,
            "Away" :0.38015420717398596
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.9077809798270892,
            "won" :0.09221902017291081
        }
    ],
    "frozen" :false
}
 'Y' '2nd' 'won']


#### b. India winning, batting 2nd, Ashwin not playing

In [26]:
print(model.predict_proba([None, None, 'N', '2nd', 'won']))

[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.5117630777747025,
            "Away" :0.4882369222252975
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.5586206896551724,
            "won" :0.44137931034482764
        }
    ],
    "frozen" :false
}
 'N' '2nd' 'won']


#### c) India losing, batting 2nd, Ashwin playing

In [27]:
print(model.predict_proba([None, None, 'Y', '2nd', 'lost']))

[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.6198457928260142,
            "Away" :0.38015420717398596
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.9077809798270892,
            "won" :0.09221902017291081
        }
    ],
    "frozen" :false
}
 'Y' '2nd' 'lost']


#### d) India losing, batting 2nd, Ashwin not playing

In [28]:
print(model.predict_proba([None, None, 'N', '2nd', 'lost']))

[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.0,
            "Away" :1.0
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.9077809798270892,
            "won" :0.09221902017291081
        }
    ],
    "frozen" :false
}
 'N' '2nd' 'lost']


<h3><center> Happy Coding!</center></h3>